In [ ]:
import numpy as np
from numpy import linalg as la
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
import cvxpy as cp
import random
import time
import os
from joblib import Parallel, delayed

from src.model import Nonneg_dagma, MetMulDagma, BarrierDagma
import src.utils as utils

from baselines.colide import colide_ev
from baselines.dagma_linear import DAGMA_linear
from baselines.notears import notears_linear

SEED = 10
N_CPUS = os.cpu_count() // 2

# Set CPU limit for np
# os.environ["OPENBLAS_NUM_THREADS"] = str(N_CPUS)
# os.environ["MKL_NUM_THREADS"] = str(N_CPUS)
# os.environ["NUMBA_NUM_THREADS"] = str(N_CPUS)
# os.environ["OMP_NUM_THREADS"] = str(N_CPUS)
# os.environ["NUMEXP_NUM_THREADS"] = str(N_CPUS)
# os.environ["MKL_NUM_THREADS"] = str(N_CPUS)
# os.environ["OMP_NUM_THREADS"] = str(N_CPUS)

np.random.seed(SEED)
random.seed(SEED)

In [ ]:
def get_lamb_value(n_nodes, n_samples, times=1):
    return np.sqrt(np.log(n_nodes) / n_samples) * times 

def run_exps(g, data_p, exps, thr=.2, verb=False):
    A_true, _, X = utils.simulate_sem(**data_p)
    A_true_bin = utils.to_bin(A_true, thr)
    X_std = utils.standarize(X)

    M, N = X.shape

    fidelity = 1/data_p['n_samples']*la.norm(X - X @ A_true, 'fro')**2
    fidelity_norm = 1/data_p['n_samples']*la.norm(X_std - X_std @ A_true, 'fro')**2

    print(f'{g}: Fidelity: {fidelity:.3f}  -  Fidelity (norm): {fidelity_norm:.3f}')

    shd, fscore, err, acyc, runtime = [np.zeros(len(exps))  for _ in range(5)]
    for i, exp in enumerate(exps):
        X_aux = X_std if 'standarize' in exp.keys() and exp['standarize'] else X

        args = exp['args'].copy()
        if 'fix_lamb' in exp.keys() and not exp['fix_lamb']:
            args['lamb'] = get_lamb_value(N, M, args['lamb'])

        if exp['model'] == notears_linear:
            t_i = time.time()
            A_est = notears_linear(X_aux, **args)
            t_solved = time.time() - t_i
        else:
            model = exp['model'](**exp['init']) if 'init' in exp.keys() else exp['model']()
            t_i = time.time()
            model.fit(X_aux, **args)
            t_solved = time.time() - t_i

            A_est = model.W_est

        A_est_bin = utils.to_bin(A_est, thr)
        shd[i], _, _ = utils.count_accuracy(A_true_bin, A_est_bin)
        fscore[i] = f1_score(A_true_bin.flatten(), A_est_bin.flatten())
        err[i] = utils.compute_norm_sq_err(A_true, A_est)
        acyc[i] = model.dagness(A_est) if hasattr(model, 'dagness') else 1
        runtime[i] = t_solved

        if verb:
            print(f'\t-{exp["leg"]}: shd {shd[i]}  -  err: {err[i]:.3f}  -  time: {runtime[i]:.3f}')

    return shd, fscore, err, acyc, runtime

### Logarithmic barrier DAGMA Alg

In [8]:
# ## Second set of hyperparams N = 50

# lamb = get_lamb_value(N, M, 2e-2)
# print('lamb:', lamb)
# if norm_x:
#     lamb = get_lamb_value(N, M, 2e-4)
#     args = {'stepsize': 5e-1, 'alpha': 5e-3, 's': 1, 'lamb': lamb, 'iters_in': 5000,
#             'iters_out': 10, 'tol': 1e-6, 'beta': .5, 'verb': True}
# else:
#     lamb = get_lamb_value(N, M, 2e-2)
#     args = {'stepsize': 2e-2, 'alpha': .01, 's': 1, 'lamb': lamb, 'iters_in': 100,
#             'iters_out': 100, 'tol': 1e-6, 'beta': .9, 'verb': True}


# t_i = time.time()
# model = BarrierDagma()
# A_est = model.fit(X, **args, track_seq=True)
# t_solved = time.time() - t_i
# print(f'Solved in {t_solved:3f} seconds')

# # Plot description
# thr = .2
# plot_convergence(model, A)
# plot_result(A, A_est, thr)

# Comparing models - Unit variance

In [9]:
Exps = [
    # Proj Grad Desc - Logdet
    {'model': Nonneg_dagma, 'args': {'stepsize': 5e-3, 'alpha': .5, 's': 1, 'lamb': .1, 'max_iters': 20000, 'tol': 1e-5},
     'init': {'acyclicity': 'logdet', 'primal_opt': 'fista'}, 'fix_lamb': False, 'standarize': False, 'leg': 'PGD-FISTA'},

    {'model': Nonneg_dagma, 'args': {'stepsize': 5e-3, 'alpha': .5, 's': 1, 'lamb': .1, 'max_iters': 20000, 'tol': 1e-5},
     'init': {'acyclicity': 'logdet', 'primal_opt': 'adam'}, 'fix_lamb': False, 'standarize': False, 'leg': 'PGD-Adam'},

    # Method of Multipliers - Logdet
    {'model': MetMulDagma, 'args': {'stepsize': 3e-4, 'alpha_0': .01, 'rho_0': .05, 's': 1, 'lamb': .1, 'iters_in': 40000,
     'iters_out': 10, 'beta': 2}, 'init': {'acyclicity': 'logdet', 'primal_opt': 'pgd'}, 'standarize': False, 
     'fix_lamb': False, 'leg': 'MM'},
    
    {'model': MetMulDagma, 'args': {'stepsize': 5e-5, 'alpha_0': .5, 'rho_0': .5, 's': 1, 'lamb': .1, 'iters_in': 20000,
     'iters_out': 10, 'beta': 2}, 'init': {'acyclicity': 'logdet', 'primal_opt': 'fista'}, 'standarize': False,
     'fix_lamb': False, 'leg': 'MM-FISTA'},

    {'model': MetMulDagma, 'args': {'stepsize': 3e-4, 'alpha_0': .01, 'rho_0': .05, 's': 1, 'lamb': .1, 'iters_in': 10000,
     'iters_out': 10, 'beta': 2}, 'init': {'acyclicity': 'logdet', 'primal_opt': 'adam'}, 'standarize': False,
     'fix_lamb': False, 'leg': 'MM-adam'},

    {'model': MetMulDagma, 'args': {'stepsize': 3e-4, 'alpha_0': .01, 'rho_0': .05, 's': 1, 'lamb': .05, 'iters_in': 10000,
     'iters_out': 10, 'beta': 2}, 'init': {'acyclicity': 'logdet', 'primal_opt': 'adam'}, 'standarize': False,
     'fix_lamb': True, 'leg': 'MM-adam-fix'},

    # Proj Grad Desc - Mat exp
    {'model': Nonneg_dagma, 'args': {'stepsize': 5e-6, 'alpha': 1e5, 's': 1, 'lamb': 2e-4, 'max_iters': 20000, 'tol': 1e-5},
     'init': {'acyclicity': 'matexp', 'primal_opt': 'fista'}, 'fix_lamb': False, 'standarize': False, 'leg': 'PGD-matexp'},

    # Method of Multipliers - Mat exp
    {'model': MetMulDagma, 'args': {'stepsize': 1e-5, 'alpha_0': 1, 'rho_0': 10, 's': 1, 'lamb': .05, 'iters_in': 10000,
     'iters_out': 5, 'beta': 10, 'tol': 1e-4}, 'init': {'acyclicity': 'matexp', 'primal_opt': 'fista'}, 'standarize': False,
     'fix_lamb': True, 'leg': 'MM-fista-matexp'},

    # Colide
    {'model': colide_ev, 'args': {'lambda1': .05, 'T': 4, 's': [1.0, .9, .8, .7], 'warm_iter': 2e4,
     'max_iter': 7e4, 'lr': .0003}, 'standarize': False, 'leg': 'CoLiDe'},

    # DAGMA
    {'model': DAGMA_linear, 'init': {'loss_type': 'l2'}, 'args': {'lambda1': .05, 'T': 4, 's': [1.0, .9, .8, .8],
     'warm_iter': 2e4, 'max_iter': 7e4, 'lr': .0003}, 'standarize': False, 'leg': 'DAGMA'},

    # NoTears
    {'model': notears_linear, 'args': {'loss_type': 'l2', 'lambda1': .1, 'max_iter': 100}, 'standarize': False, 'leg': 'NoTears'},
]

## 4N Edges

In [10]:
N = 100

n_dags = 30
verb = True
data_params = {
    'n_nodes': N,
    'n_samples': 500, # 1000,
    'graph_type': 'er',
    'edges': 4*N,
    'edge_type': 'positive',
    'w_range': (.5, 1),
    'var': 1
}

print('CPUs employed:', N_CPUS)

t_init = time.time()
results = Parallel(n_jobs=N_CPUS)(delayed(run_exps)(g, data_params, Exps, thr=.2, verb=verb) for g in range(n_dags))
ellapsed_time = (time.time() - t_init)/60
print(f'----- Solved in {ellapsed_time:.3f} minutes -----')

# Extract results
shd, fscore, err, acyc, runtime = zip(*results)
metrics = {'shd': shd, 'fscore': fscore, 'err': err, 'acyc': acyc, 'time': runtime}


CPUs employed: 32
13: Fidelity: 100.838  -  Fidelity (norm): 329.363
5: Fidelity: 100.053  -  Fidelity (norm): 449.676
6: Fidelity: 100.519  -  Fidelity (norm): 275.233
12: Fidelity: 100.626  -  Fidelity (norm): 485.233
3: Fidelity: 99.757  -  Fidelity (norm): 237.474
2: Fidelity: 99.369  -  Fidelity (norm): 434.859
7: Fidelity: 100.147  -  Fidelity (norm): 510.567
15: Fidelity: 99.269  -  Fidelity (norm): 285.799
29: Fidelity: 100.657  -  Fidelity (norm): 438.890
10: Fidelity: 99.606  -  Fidelity (norm): 347.126
4: Fidelity: 100.634  -  Fidelity (norm): 317.059
17: Fidelity: 100.608  -  Fidelity (norm): 428.193
11: Fidelity: 99.554  -  Fidelity (norm): 328.621
25: Fidelity: 99.621  -  Fidelity (norm): 405.389
9: Fidelity: 99.972  -  Fidelity (norm): 388.106
1: Fidelity: 99.360  -  Fidelity (norm): 261.160
14: Fidelity: 101.212  -  Fidelity (norm): 419.305
23: Fidelity: 100.506  -  Fidelity (norm): 287.018
0: Fidelity: 100.728  -  Fidelity (norm): 355.449
22: Fidelity: 99.181  -  Fidel

KeyboardInterrupt: 

In [ ]:
exps_leg = [exp['leg'] for exp in Exps]
utils.display_results(exps_leg, metrics, agg='mean')
utils.display_results(exps_leg, metrics, agg='median')

,leg,shd,fscore,err,acyc,time
0,PGD-FISTA,44.4667 ± 113.7663,0.9300 ± 0.1799,0.1456 ± 0.3455,1.4809 ± 0.4419,9.5156 ± 2.7243
1,PGD-Adam,0.1667 ± 0.5821,0.9997 ± 0.0013,0.0125 ± 0.0012,1.6319 ± 0.2605,12.5988 ± 3.4132
2,MM,47.9667 ± 68.9630,0.9396 ± 0.0842,0.1166 ± 0.1506,0.0075 ± 0.0238,246.0834 ± 106.3629
3,MM-FISTA,48.6000 ± 92.1041,0.9361 ± 0.1270,0.0987 ± 0.1895,0.0764 ± 0.2068,27.7045 ± 13.0354
4,MM-adam,0.2333 ± 0.7608,0.9996 ± 0.0014,0.0103 ± 0.0014,0.0020 ± 0.0007,35.2052 ± 10.1880
5,MM-adam-fix,2.1333 ± 2.1868,0.9967 ± 0.0038,0.0174 ± 0.0043,0.0121 ± 0.0035,29.8664 ± 10.0082
6,CoLiDe,32.3667 ± 14.1197,0.9489 ± 0.0201,0.0803 ± 0.0296,0.0003 ± 0.0000,66.1110 ± 27.7100
7,DAGMA,29.2667 ± 15.6481,0.9535 ± 0.0220,0.0773 ± 0.0293,1.0000 ± 0.0000,50.3380 ± 18.2213
8,NoTears,57.1000 ± 23.0570,0.9152 ± 0.0316,0.1466 ± 0.0538,1.0000 ± 0.0000,886.4303 ± 213.3939


,leg,shd,fscore,err,acyc,time
0,PGD-FISTA,0.0000 ± 113.7663,1.0000 ± 0.1799,0.0125 ± 0.3455,1.5558 ± 0.4419,9.3453 ± 2.7243
1,PGD-Adam,0.0000 ± 0.5821,1.0000 ± 0.0013,0.0123 ± 0.0012,1.6349 ± 0.2605,14.6472 ± 3.4132
2,MM,16.0000 ± 68.9630,0.9743 ± 0.0842,0.0548 ± 0.1506,0.0015 ± 0.0238,236.3029 ± 106.3629
3,MM-FISTA,1.5000 ± 92.1041,0.9969 ± 0.1270,0.0127 ± 0.1895,0.0041 ± 0.2068,22.5085 ± 13.0354
4,MM-adam,0.0000 ± 0.7608,1.0000 ± 0.0014,0.0100 ± 0.0014,0.0019 ± 0.0007,35.5078 ± 10.1880
5,MM-adam-fix,1.0000 ± 2.1868,0.9982 ± 0.0038,0.0161 ± 0.0043,0.0126 ± 0.0035,26.7692 ± 10.0082
6,CoLiDe,30.5000 ± 14.1197,0.9523 ± 0.0201,0.0755 ± 0.0296,0.0003 ± 0.0000,55.4217 ± 27.7100
7,DAGMA,27.5000 ± 15.6481,0.9538 ± 0.0220,0.0765 ± 0.0293,1.0000 ± 0.0000,46.4685 ± 18.2213
8,NoTears,58.5000 ± 23.0570,0.9104 ± 0.0316,0.1504 ± 0.0538,1.0000 ± 0.0000,953.6134 ± 213.3939


## 2N Edges - 100 nodes

In [ ]:
###  1000 samples - 100 nodes  #####
N = 100

n_dags = 25
data_params = {
    'n_nodes': N,
    'n_samples': 500, # 1000,
    'graph_type': 'er',
    'edges': 2*N,
    'edge_type': 'positive',
    'w_range': (.5, 1),
    'var': 1,
}

print('CPUs employed:', N_CPUS)

t_init = time.time()
results = Parallel(n_jobs=N_CPUS)(delayed(run_exps)(g, data_params, Exps, thr=.2, verb=False) for g in range(n_dags))
ellapsed_time = (time.time() - t_init)/60
print(f'----- Solved in {ellapsed_time:.3f} minutes -----')

# Extract results
shd, fscore, err, acyc, runtime = zip(*results)
metrics = {'shd': shd, 'fscore': fscore, 'err': err, 'acyc': acyc, 'time': runtime}


CPUs employed: 32
0: Fidelity: 99.166  -  Fidelity (norm): 83.547
1: Fidelity: 99.166  -  Fidelity (norm): 74.474
2: Fidelity: 99.166  -  Fidelity (norm): 86.251
3: Fidelity: 99.166  -  Fidelity (norm): 79.338
4: Fidelity: 99.166  -  Fidelity (norm): 94.518
5: Fidelity: 99.166  -  Fidelity (norm): 74.226
6: Fidelity: 99.166  -  Fidelity (norm): 82.351
7: Fidelity: 99.166  -  Fidelity (norm): 85.536
8: Fidelity: 99.166  -  Fidelity (norm): 85.159
10: Fidelity: 99.166  -  Fidelity (norm): 86.291
9: Fidelity: 99.166  -  Fidelity (norm): 78.503
11: Fidelity: 99.166  -  Fidelity (norm): 78.843
24: Fidelity: 100.084  -  Fidelity (norm): 83.375
21: Fidelity: 100.867  -  Fidelity (norm): 74.127
23: Fidelity: 100.550  -  Fidelity (norm): 97.654
19: Fidelity: 100.058  -  Fidelity (norm): 71.361
18: Fidelity: 100.548  -  Fidelity (norm): 98.949
16: Fidelity: 99.614  -  Fidelity (norm): 83.142
13: Fidelity: 100.605  -  Fidelity (norm): 73.057
12: Fidelity: 100.169  -  Fidelity (norm): 86.856
14: F

  0%|          | 0/130000.0 [00:18<?, ?it/s]
  0%|          | 0/130000.0 [00:18<?, ?it/s]
  0%|          | 0/130000.0 [00:21<?, ?it/s]
  0%|          | 0/130000.0 [00:20<?, ?it/s]
  0%|          | 0/130000.0 [00:28<?, ?it/s]
  0%|          | 0/130000.0 [00:34<?, ?it/s]
  0%|          | 0/130000.0 [00:30<?, ?it/s]
  0%|          | 0/130000.0 [00:33<?, ?it/s]
  0%|          | 0/130000.0 [00:32<?, ?it/s]
  0%|          | 0/130000.0 [00:32<?, ?it/s]
  0%|          | 0/130000.0 [00:34<?, ?it/s]
  0%|          | 0/130000.0 [00:37<?, ?it/s]
  0%|          | 0/130000.0 [00:34<?, ?it/s]
  0%|          | 0/130000.0 [00:34<?, ?it/s]
  0%|          | 0/130000.0 [00:37<?, ?it/s]
  0%|          | 0/130000.0 [00:39<?, ?it/s]
  0%|          | 0/130000.0 [00:31<?, ?it/s]
  0%|          | 0/130000.0 [00:35<?, ?it/s]
  0%|          | 0/130000.0 [00:42<?, ?it/s]
  0%|          | 0/130000.0 [00:43<?, ?it/s]
  0%|          | 0/130000.0 [00:35<?, ?it/s]
  0%|          | 0/130000.0 [00:41<?, ?it/s]
  0%|     

----- Solved in 4.87e+02 seconds -----


In [ ]:
# M = 1000
exps_leg = [exp['leg'] for exp in Exps]
utils.display_results(exps_leg, metrics, agg='mean')
utils.display_results(exps_leg, metrics, agg='median')

,leg,shd,fscore,err,acyc,time
0,PGD-FISTA,0.1600 ± 0.4630,0.9995 ± 0.0015,0.0210 ± 0.0084,0.7234 ± 0.3196,2.5151 ± 0.9965
1,PGD-Adam,0.3200 ± 0.6145,0.9986 ± 0.0027,0.0235 ± 0.0043,0.4926 ± 0.2934,2.5296 ± 0.7141
2,MM,0.2000 ± 0.5657,0.9992 ± 0.0023,0.0169 ± 0.0019,0.0072 ± 0.0009,72.5910 ± 10.2621
3,MM-FISTA,15.2000 ± 38.6937,0.9661 ± 0.0814,0.0685 ± 0.1323,0.0119 ± 0.0205,21.2054 ± 7.2162
4,MM-adam,0.7200 ± 2.0692,0.9975 ± 0.0062,0.0178 ± 0.0059,0.0052 ± 0.0007,17.7671 ± 3.9768
5,MM-adam-fix,2.1200 ± 2.8750,0.9914 ± 0.0085,0.0192 ± 0.0102,0.0084 ± 0.0012,15.1349 ± 3.2953
6,CoLiDe,6.1600 ± 4.2772,0.9783 ± 0.0133,0.0366 ± 0.0161,0.0001 ± 0.0000,33.4869 ± 7.2743
7,DAGMA,5.4400 ± 4.2622,0.9809 ± 0.0133,0.0348 ± 0.0160,1.0000 ± 0.0000,33.1552 ± 7.6967
8,NoTears,5.2000 ± 3.1241,0.9805 ± 0.0101,0.0330 ± 0.0149,1.0000 ± 0.0000,186.8967 ± 43.4586


,leg,shd,fscore,err,acyc,time
0,PGD-FISTA,0.0000 ± 0.4630,1.0000 ± 0.0015,0.0187 ± 0.0084,0.7474 ± 0.3196,2.0837 ± 0.9965
1,PGD-Adam,0.0000 ± 0.6145,1.0000 ± 0.0027,0.0233 ± 0.0043,0.5255 ± 0.2934,2.4221 ± 0.7141
2,MM,0.0000 ± 0.5657,1.0000 ± 0.0023,0.0166 ± 0.0019,0.0069 ± 0.0009,73.7634 ± 10.2621
3,MM-FISTA,0.0000 ± 38.6937,1.0000 ± 0.0814,0.0174 ± 0.1323,0.0062 ± 0.0205,19.5585 ± 7.2162
4,MM-adam,0.0000 ± 2.0692,1.0000 ± 0.0062,0.0161 ± 0.0059,0.0051 ± 0.0007,16.7323 ± 3.9768
5,MM-adam-fix,2.0000 ± 2.8750,0.9929 ± 0.0085,0.0147 ± 0.0102,0.0085 ± 0.0012,14.2340 ± 3.2953
6,CoLiDe,5.0000 ± 4.2772,0.9804 ± 0.0133,0.0356 ± 0.0161,0.0001 ± 0.0000,34.6268 ± 7.2743
7,DAGMA,4.0000 ± 4.2622,0.9839 ± 0.0133,0.0316 ± 0.0160,1.0000 ± 0.0000,33.5630 ± 7.6967
8,NoTears,5.0000 ± 3.1241,0.9822 ± 0.0101,0.0320 ± 0.0149,1.0000 ± 0.0000,185.9129 ± 43.4586


,leg,shd,fscore,err,acyc,time
0,PGD-FISTA,0.1600 ± 0.4630,0.9995 ± 0.0015,0.0210 ± 0.0084,0.7234 ± 0.3196,2.5151 ± 0.9965
1,PGD-Adam,0.3200 ± 0.6145,0.9986 ± 0.0027,0.0235 ± 0.0043,0.4926 ± 0.2934,2.5296 ± 0.7141
2,MM,0.2000 ± 0.5657,0.9992 ± 0.0023,0.0169 ± 0.0019,0.0072 ± 0.0009,72.5910 ± 10.2621
3,MM-FISTA,15.2000 ± 38.6937,0.9661 ± 0.0814,0.0685 ± 0.1323,0.0119 ± 0.0205,21.2054 ± 7.2162
4,MM-adam,0.7200 ± 2.0692,0.9975 ± 0.0062,0.0178 ± 0.0059,0.0052 ± 0.0007,17.7671 ± 3.9768
5,MM-adam-fix,2.1200 ± 2.8750,0.9914 ± 0.0085,0.0192 ± 0.0102,0.0084 ± 0.0012,15.1349 ± 3.2953
6,CoLiDe,6.1600 ± 4.2772,0.9783 ± 0.0133,0.0366 ± 0.0161,0.0001 ± 0.0000,33.4869 ± 7.2743
7,DAGMA,5.4400 ± 4.2622,0.9809 ± 0.0133,0.0348 ± 0.0160,1.0000 ± 0.0000,33.1552 ± 7.6967
8,NoTears,5.2000 ± 3.1241,0.9805 ± 0.0101,0.0330 ± 0.0149,1.0000 ± 0.0000,186.8967 ± 43.4586


# Comparing models - Standarized

In [ ]:
Exps = [
    # Proj Grad Desc
    {'model': Nonneg_dagma, 'args': {'stepsize': 5e-3, 'alpha': .5, 's': 1, 'lamb': .1, 'max_iters': 20000, 'tol': 1e-5},
     'init': {'acyclicity': 'logdet', 'primal_opt': 'fista'}, 'fix_lamb': False, 'standarize': True, 'leg': 'PGD-FISTA'},

    {'model': Nonneg_dagma, 'args': {'stepsize': 5e-3, 'alpha': .5, 's': 1, 'lamb': .1, 'max_iters': 20000, 'tol': 1e-5},
     'init': {'acyclicity': 'logdet', 'primal_opt': 'adam'}, 'fix_lamb': False, 'standarize': True, 'leg': 'PGD-Adam'},

    # Method of Multipliers
    {'model': MetMulDagma, 'args': {'stepsize': 3e-4, 'alpha_0': .01, 'rho_0': .05, 's': 1, 'lamb': .1, 'iters_in': 40000,
     'iters_out': 10, 'beta': 2}, 'init': {'acyclicity': 'logdet', 'primal_opt': 'pgd'}, 'standarize': True, 
     'fix_lamb': False, 'leg': 'MM'},

    {'model': MetMulDagma, 'args': {'stepsize': 5e-5, 'alpha_0': .5, 'rho_0': .5, 's': 1, 'lamb': .1, 'iters_in': 20000,
     'iters_out': 10, 'beta': 2}, 'init': {'acyclicity': 'logdet', 'primal_opt': 'fista'}, 'standarize': True,
     'fix_lamb': False, 'leg': 'MM-FISTA'},

    {'model': MetMulDagma, 'args': {'stepsize': 3e-4, 'alpha_0': .01, 'rho_0': .05, 's': 1, 'lamb': .1, 'iters_in': 10000,
     'iters_out': 10, 'beta': 2}, 'init': {'acyclicity': 'logdet', 'primal_opt': 'adam'}, 'standarize': True,
     'fix_lamb': False, 'leg': 'MM-adam'},

    {'model': MetMulDagma, 'args': {'stepsize': 3e-4, 'alpha_0': .01, 'rho_0': .05, 's': 1, 'lamb': .05, 'iters_in': 10000,
     'iters_out': 10, 'beta': 2}, 'init': {'acyclicity': 'logdet', 'primal_opt': 'adam'}, 'standarize': True,
     'fix_lamb': True, 'leg': 'MM-adam-fix'},

    # Proj Grad Desc - Mat exp
    {'model': Nonneg_dagma, 'args': {'stepsize': 5e-6, 'alpha': 1e5, 's': 1, 'lamb': 2e-4, 'max_iters': 20000, 'tol': 1e-5},
     'init': {'acyclicity': 'matexp', 'primal_opt': 'fista'}, 'fix_lamb': False, 'standarize': True, 'leg': 'PGD-matexp'},

    # Method of Multipliers - Mat exp
    {'model': MetMulDagma, 'args': {'stepsize': 1e-4, 'alpha_0': 1, 'rho_0': 10, 's': 1, 'lamb': .05, 'iters_in': 10000,
     'iters_out': 5, 'beta': 10, 'tol': 1e-4}, 'init': {'acyclicity': 'matexp', 'primal_opt': 'fista'}, 'standarize': True,
     'fix_lamb': True, 'leg': 'MM-fista-matexp'},

    # Colide
    {'model': colide_ev, 'args': {'lambda1': .05, 'T': 4, 's': [1.0, .9, .8, .7], 'warm_iter': 2e4,
     'max_iter': 7e4, 'lr': .0003}, 'standarize': True, 'leg': 'CoLiDe'},

    # DAGMA
    {'model': DAGMA_linear, 'init': {'loss_type': 'l2'}, 'args': {'lambda1': .05, 'T': 4, 's': [1.0, .9, .8, .8],
     'warm_iter': 2e4, 'max_iter': 7e4, 'lr': .0003}, 'standarize': True, 'leg': 'DAGMA'},

    # NoTears
    {'model': notears_linear, 'args': {'loss_type': 'l2', 'lambda1': .1, 'max_iter': 100}, 'standarize': True, 'leg': 'NoTears'},
]

## 4N Edges

In [ ]:
N = 100

n_dags = 30
verb=True
data_params = {
    'n_nodes': N,
    'n_samples': 500, # 1000,
    'graph_type': 'er',
    'edges': 4*N,
    'edge_type': 'positive',
    'w_range': (.5, 1),
    'var': 1
}

print('CPUs employed:', N_CPUS)

t_init = time.time()
results = Parallel(n_jobs=N_CPUS)(delayed(run_exps)(g, data_params, Exps, thr=.2, verb=verb) for g in range(n_dags))
ellapsed_time = (time.time() - t_init)/60
print(f'----- Solved in {ellapsed_time:.3f} seminutesconds -----')

# Extract results
shd, fscore, err, acyc, runtime = zip(*results)
metrics = {'shd': shd, 'fscore': fscore, 'err': err, 'acyc': acyc, 'time': runtime}


CPUs employed: 32
26: Fidelity: 100.350  -  Fidelity (norm): 360.430
25: Fidelity: 99.762  -  Fidelity (norm): 266.280
16: Fidelity: 99.354  -  Fidelity (norm): 338.878
10: Fidelity: 99.507  -  Fidelity (norm): 357.485
11: Fidelity: 99.970  -  Fidelity (norm): 251.609
24: Fidelity: 99.867  -  Fidelity (norm): 335.004
14: Fidelity: 98.683  -  Fidelity (norm): 384.531
18: Fidelity: 99.427  -  Fidelity (norm): 438.379
5: Fidelity: 99.511  -  Fidelity (norm): 459.561
4: Fidelity: 98.891  -  Fidelity (norm): 384.318
7: Fidelity: 100.306  -  Fidelity (norm): 347.501
28: Fidelity: 102.664  -  Fidelity (norm): 378.917
19: Fidelity: 100.862  -  Fidelity (norm): 439.882
1: Fidelity: 99.975  -  Fidelity (norm): 372.822
0: Fidelity: 100.057  -  Fidelity (norm): 516.286
21: Fidelity: 100.559  -  Fidelity (norm): 371.004
22: Fidelity: 100.104  -  Fidelity (norm): 321.517
9: Fidelity: 99.791  -  Fidelity (norm): 279.769
23: Fidelity: 100.332  -  Fidelity (norm): 376.308
17: Fidelity: 99.653  -  Fidel

  0%|          | 0/130000.0 [00:00<?, ?it/s]

	-MM-FISTA: shd 56.0  -  err: 0.080  -  time: 27.074
	-MM: shd 0.0  -  err: 0.013  -  time: 102.653
	-MM-adam-fix: shd 1.0  -  err: 0.013  -  time: 15.490


  0%|          | 0/130000.0 [00:00<?, ?it/s]

	-MM-adam: shd 0.0  -  err: 0.010  -  time: 20.456
	-MM-FISTA: shd 6.0  -  err: 0.018  -  time: 20.712


...

	-CoLiDe: shd 44.0  -  err: 0.088  -  time: 113.347
	-DAGMA: shd 14.0  -  err: 0.043  -  time: 56.028
	-DAGMA: shd 36.0  -  err: 0.094  -  time: 78.480


  0%|          | 0/130000.0 [01:25<?, ?it/s]


	-CoLiDe: shd 30.0  -  err: 0.088  -  time: 85.016
	-DAGMA: shd 42.0  -  err: 0.102  -  time: 47.084
	-DAGMA: shd 44.0  -  err: 0.093  -  time: 80.279


  0%|          | 0/130000.0 [02:41<?, ?it/s]


	-CoLiDe: shd 44.0  -  err: 0.097  -  time: 161.493
	-DAGMA: shd 28.0  -  err: 0.091  -  time: 72.034
	-DAGMA: shd 70.0  -  err: 0.134  -  time: 122.318
	-NoTears: shd 31.0  -  err: 0.085  -  time: 553.880
	-NoTears: shd 55.0  -  err: 0.141  -  time: 698.109
	-NoTears: shd 28.0  -  err: 0.082  -  time: 660.270
	-NoTears: shd 24.0  -  err: 0.067  -  time: 789.013
	-NoTears: shd 23.0  -  err: 0.071  -  time: 707.888
	-NoTears: shd 77.0  -  err: 0.180  -  time: 775.003
	-NoTears: shd 37.0  -  err: 0.093  -  time: 677.460
	-NoTears: shd 41.0  -  err: 0.081  -  time: 846.150
	-NoTears: shd 47.0  -  err: 0.137  -  time: 1080.165
	-NoTears: shd 57.0  -  err: 0.152  -  time: 891.720
	-NoTears: shd 70.0  -  err: 0.159  -  time: 778.274
	-NoTears: shd 51.0  -  err: 0.134  -  time: 1069.880
	-NoTears: shd 63.0  -  err: 0.176  -  time: 954.576
	-NoTears: shd 69.0  -  err: 0.174  -  time: 840.213
	-NoTears: shd 87.0  -  err: 0.229  -  time: 1072.547
	-NoTears: shd 75.0  -  err: 0.190  -  time: 1065

In [ ]:
exps_leg = [exp['leg'] for exp in Exps]
utils.display_results(exps_leg, metrics, agg='mean')
utils.display_results(exps_leg, metrics, agg='median')

,leg,shd,fscore,err,acyc,time
0,PGD-FISTA,44.4667 ± 113.7663,0.9300 ± 0.1799,0.1456 ± 0.3455,1.4809 ± 0.4419,9.5156 ± 2.7243
1,PGD-Adam,0.1667 ± 0.5821,0.9997 ± 0.0013,0.0125 ± 0.0012,1.6319 ± 0.2605,12.5988 ± 3.4132
2,MM,47.9667 ± 68.9630,0.9396 ± 0.0842,0.1166 ± 0.1506,0.0075 ± 0.0238,246.0834 ± 106.3629
3,MM-FISTA,48.6000 ± 92.1041,0.9361 ± 0.1270,0.0987 ± 0.1895,0.0764 ± 0.2068,27.7045 ± 13.0354
4,MM-adam,0.2333 ± 0.7608,0.9996 ± 0.0014,0.0103 ± 0.0014,0.0020 ± 0.0007,35.2052 ± 10.1880
5,MM-adam-fix,2.1333 ± 2.1868,0.9967 ± 0.0038,0.0174 ± 0.0043,0.0121 ± 0.0035,29.8664 ± 10.0082
6,CoLiDe,32.3667 ± 14.1197,0.9489 ± 0.0201,0.0803 ± 0.0296,0.0003 ± 0.0000,66.1110 ± 27.7100
7,DAGMA,29.2667 ± 15.6481,0.9535 ± 0.0220,0.0773 ± 0.0293,1.0000 ± 0.0000,50.3380 ± 18.2213
8,NoTears,57.1000 ± 23.0570,0.9152 ± 0.0316,0.1466 ± 0.0538,1.0000 ± 0.0000,886.4303 ± 213.3939


,leg,shd,fscore,err,acyc,time
0,PGD-FISTA,0.0000 ± 113.7663,1.0000 ± 0.1799,0.0125 ± 0.3455,1.5558 ± 0.4419,9.3453 ± 2.7243
1,PGD-Adam,0.0000 ± 0.5821,1.0000 ± 0.0013,0.0123 ± 0.0012,1.6349 ± 0.2605,14.6472 ± 3.4132
2,MM,16.0000 ± 68.9630,0.9743 ± 0.0842,0.0548 ± 0.1506,0.0015 ± 0.0238,236.3029 ± 106.3629
3,MM-FISTA,1.5000 ± 92.1041,0.9969 ± 0.1270,0.0127 ± 0.1895,0.0041 ± 0.2068,22.5085 ± 13.0354
4,MM-adam,0.0000 ± 0.7608,1.0000 ± 0.0014,0.0100 ± 0.0014,0.0019 ± 0.0007,35.5078 ± 10.1880
5,MM-adam-fix,1.0000 ± 2.1868,0.9982 ± 0.0038,0.0161 ± 0.0043,0.0126 ± 0.0035,26.7692 ± 10.0082
6,CoLiDe,30.5000 ± 14.1197,0.9523 ± 0.0201,0.0755 ± 0.0296,0.0003 ± 0.0000,55.4217 ± 27.7100
7,DAGMA,27.5000 ± 15.6481,0.9538 ± 0.0220,0.0765 ± 0.0293,1.0000 ± 0.0000,46.4685 ± 18.2213
8,NoTears,58.5000 ± 23.0570,0.9104 ± 0.0316,0.1504 ± 0.0538,1.0000 ± 0.0000,953.6134 ± 213.3939


In [ ]:
###  1000 samples - 100 nodes  #####
N = 100

n_dags = 25
data_params = {
    'n_nodes': N,
    'n_samples': 500, # 1000,
    'graph_type': 'er',
    'edges': 2*N,
    'edge_type': 'positive',
    'w_range': (.5, 1),
    'var': 1,
}

print('CPUs employed:', N_CPUS)

t_init = time.time()
results = Parallel(n_jobs=N_CPUS)(delayed(run_exps)(g, data_params, Exps, thr=.2, verb=False) for g in range(n_dags))
ellapsed_time = (time.time() - t_init)/60
print(f'----- Solved in {ellapsed_time:.3f} minutes -----')

# Extract results
shd, fscore, err, acyc, runtime = zip(*results)
metrics = {'shd': shd, 'fscore': fscore, 'err': err, 'acyc': acyc, 'time': runtime}


CPUs employed: 32
0: Fidelity: 99.166  -  Fidelity (norm): 83.547
1: Fidelity: 99.166  -  Fidelity (norm): 74.474
2: Fidelity: 99.166  -  Fidelity (norm): 86.251
3: Fidelity: 99.166  -  Fidelity (norm): 79.338
4: Fidelity: 99.166  -  Fidelity (norm): 94.518
5: Fidelity: 99.166  -  Fidelity (norm): 74.226
6: Fidelity: 99.166  -  Fidelity (norm): 82.351
7: Fidelity: 99.166  -  Fidelity (norm): 85.536
8: Fidelity: 99.166  -  Fidelity (norm): 85.159
10: Fidelity: 99.166  -  Fidelity (norm): 86.291
9: Fidelity: 99.166  -  Fidelity (norm): 78.503
11: Fidelity: 99.166  -  Fidelity (norm): 78.843
24: Fidelity: 100.084  -  Fidelity (norm): 83.375
21: Fidelity: 100.867  -  Fidelity (norm): 74.127
23: Fidelity: 100.550  -  Fidelity (norm): 97.654
19: Fidelity: 100.058  -  Fidelity (norm): 71.361
18: Fidelity: 100.548  -  Fidelity (norm): 98.949
16: Fidelity: 99.614  -  Fidelity (norm): 83.142
13: Fidelity: 100.605  -  Fidelity (norm): 73.057
12: Fidelity: 100.169  -  Fidelity (norm): 86.856
14: F

  0%|          | 0/130000.0 [00:18<?, ?it/s]
  0%|          | 0/130000.0 [00:18<?, ?it/s]
  0%|          | 0/130000.0 [00:21<?, ?it/s]
  0%|          | 0/130000.0 [00:20<?, ?it/s]
  0%|          | 0/130000.0 [00:28<?, ?it/s]
  0%|          | 0/130000.0 [00:34<?, ?it/s]
  0%|          | 0/130000.0 [00:30<?, ?it/s]
  0%|          | 0/130000.0 [00:33<?, ?it/s]
  0%|          | 0/130000.0 [00:32<?, ?it/s]
  0%|          | 0/130000.0 [00:32<?, ?it/s]
  0%|          | 0/130000.0 [00:34<?, ?it/s]
  0%|          | 0/130000.0 [00:37<?, ?it/s]
  0%|          | 0/130000.0 [00:34<?, ?it/s]
  0%|          | 0/130000.0 [00:34<?, ?it/s]
  0%|          | 0/130000.0 [00:37<?, ?it/s]
  0%|          | 0/130000.0 [00:39<?, ?it/s]
  0%|          | 0/130000.0 [00:31<?, ?it/s]
  0%|          | 0/130000.0 [00:35<?, ?it/s]
  0%|          | 0/130000.0 [00:42<?, ?it/s]
  0%|          | 0/130000.0 [00:43<?, ?it/s]
  0%|          | 0/130000.0 [00:35<?, ?it/s]
  0%|          | 0/130000.0 [00:41<?, ?it/s]
  0%|     

----- Solved in 4.87e+02 seconds -----


In [ ]:
# M = 1000
exps_leg = [exp['leg'] for exp in Exps]
utils.display_results(exps_leg, metrics, agg='mean')
utils.display_results(exps_leg, metrics, agg='median')

,leg,shd,fscore,err,acyc,time
0,PGD-FISTA,0.1600 ± 0.4630,0.9995 ± 0.0015,0.0210 ± 0.0084,0.7234 ± 0.3196,2.5151 ± 0.9965
1,PGD-Adam,0.3200 ± 0.6145,0.9986 ± 0.0027,0.0235 ± 0.0043,0.4926 ± 0.2934,2.5296 ± 0.7141
2,MM,0.2000 ± 0.5657,0.9992 ± 0.0023,0.0169 ± 0.0019,0.0072 ± 0.0009,72.5910 ± 10.2621
3,MM-FISTA,15.2000 ± 38.6937,0.9661 ± 0.0814,0.0685 ± 0.1323,0.0119 ± 0.0205,21.2054 ± 7.2162
4,MM-adam,0.7200 ± 2.0692,0.9975 ± 0.0062,0.0178 ± 0.0059,0.0052 ± 0.0007,17.7671 ± 3.9768
5,MM-adam-fix,2.1200 ± 2.8750,0.9914 ± 0.0085,0.0192 ± 0.0102,0.0084 ± 0.0012,15.1349 ± 3.2953
6,CoLiDe,6.1600 ± 4.2772,0.9783 ± 0.0133,0.0366 ± 0.0161,0.0001 ± 0.0000,33.4869 ± 7.2743
7,DAGMA,5.4400 ± 4.2622,0.9809 ± 0.0133,0.0348 ± 0.0160,1.0000 ± 0.0000,33.1552 ± 7.6967
8,NoTears,5.2000 ± 3.1241,0.9805 ± 0.0101,0.0330 ± 0.0149,1.0000 ± 0.0000,186.8967 ± 43.4586


,leg,shd,fscore,err,acyc,time
0,PGD-FISTA,0.0000 ± 0.4630,1.0000 ± 0.0015,0.0187 ± 0.0084,0.7474 ± 0.3196,2.0837 ± 0.9965
1,PGD-Adam,0.0000 ± 0.6145,1.0000 ± 0.0027,0.0233 ± 0.0043,0.5255 ± 0.2934,2.4221 ± 0.7141
2,MM,0.0000 ± 0.5657,1.0000 ± 0.0023,0.0166 ± 0.0019,0.0069 ± 0.0009,73.7634 ± 10.2621
3,MM-FISTA,0.0000 ± 38.6937,1.0000 ± 0.0814,0.0174 ± 0.1323,0.0062 ± 0.0205,19.5585 ± 7.2162
4,MM-adam,0.0000 ± 2.0692,1.0000 ± 0.0062,0.0161 ± 0.0059,0.0051 ± 0.0007,16.7323 ± 3.9768
5,MM-adam-fix,2.0000 ± 2.8750,0.9929 ± 0.0085,0.0147 ± 0.0102,0.0085 ± 0.0012,14.2340 ± 3.2953
6,CoLiDe,5.0000 ± 4.2772,0.9804 ± 0.0133,0.0356 ± 0.0161,0.0001 ± 0.0000,34.6268 ± 7.2743
7,DAGMA,4.0000 ± 4.2622,0.9839 ± 0.0133,0.0316 ± 0.0160,1.0000 ± 0.0000,33.5630 ± 7.6967
8,NoTears,5.0000 ± 3.1241,0.9822 ± 0.0101,0.0320 ± 0.0149,1.0000 ± 0.0000,185.9129 ± 43.4586


,leg,shd,fscore,err,acyc,time
0,PGD-FISTA,0.1600 ± 0.4630,0.9995 ± 0.0015,0.0210 ± 0.0084,0.7234 ± 0.3196,2.5151 ± 0.9965
1,PGD-Adam,0.3200 ± 0.6145,0.9986 ± 0.0027,0.0235 ± 0.0043,0.4926 ± 0.2934,2.5296 ± 0.7141
2,MM,0.2000 ± 0.5657,0.9992 ± 0.0023,0.0169 ± 0.0019,0.0072 ± 0.0009,72.5910 ± 10.2621
3,MM-FISTA,15.2000 ± 38.6937,0.9661 ± 0.0814,0.0685 ± 0.1323,0.0119 ± 0.0205,21.2054 ± 7.2162
4,MM-adam,0.7200 ± 2.0692,0.9975 ± 0.0062,0.0178 ± 0.0059,0.0052 ± 0.0007,17.7671 ± 3.9768
5,MM-adam-fix,2.1200 ± 2.8750,0.9914 ± 0.0085,0.0192 ± 0.0102,0.0084 ± 0.0012,15.1349 ± 3.2953
6,CoLiDe,6.1600 ± 4.2772,0.9783 ± 0.0133,0.0366 ± 0.0161,0.0001 ± 0.0000,33.4869 ± 7.2743
7,DAGMA,5.4400 ± 4.2622,0.9809 ± 0.0133,0.0348 ± 0.0160,1.0000 ± 0.0000,33.1552 ± 7.6967
8,NoTears,5.2000 ± 3.1241,0.9805 ± 0.0101,0.0330 ± 0.0149,1.0000 ± 0.0000,186.8967 ± 43.4586
